<a href="https://colab.research.google.com/github/nyanta012/demo/blob/main/faster_whisper_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ランタイム⇒ランタイムのタイプの変更でGPUをオンにする必要があります

In [1]:
!pip install "faster-whisper @ https://github.com/nyanta012/faster-whisper/archive/refs/heads/master.tar.gz"
!pip install transformers==4.27.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     \ 13.5 kB 167.7 kB/s 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.2/31.2 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.5 MB/s eta 0:00:00
  Created wheel for faster-whisper: filename=faster_whisper-0.1.0-py3-none-any.whl size=13988 sha256=c0dfcaed98c8b9e03def86a63ecb13c297dd13c742b8a112040a4b60446cedd9
  Stored in directory: /tmp/pip-ephem-wheel-cache-3p9z8akx/wheels/e7/6c/49/8cf262ab990e0f9835ae16806f9138d17376028f08132ec7b9
Successfully built faster-whisper
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 16.0 MB/s eta 0:00:00


In [2]:
# srt file を作成するための関数
def create_srt_file(file_name="transcribe", results=None, fast_whisper=True):
    with open(f"{file_name}.srt", mode="w") as f:
        for index, _dict in enumerate(results):
            if fast_whisper:
              start_time = _dict[0] # start
              end_time = _dict[1] # end
              text = _dict[2] # text
            else:
              start_time = _dict["start"]
              end_time = _dict["end"]
              text = _dict["text"]
            s_h, e_h = int(start_time//(60 * 60)), int(end_time//(60 * 60))
            s_m, e_m = int((start_time % (60 * 60))//60), int((end_time % (60 * 60))//60)
            s_s, e_s = int(start_time % 60), int(end_time % 60)
            f.write(f'{index+1}\n{s_h:02}:{s_m:02}:{s_s:02},000 --> {e_h:02}:{e_m:02}:{e_s:02},000\n{text}\n\n')

In [ ]:
# アップロード後、下記を実行することで、ファイル名を取得
import os
os.listdir(".")

['.config', 'sample_data']

In [ ]:
# アップロードしたファイル名を設定(拡張子まで含める)
file_name = "サンプル動画.mkv"

下記はFaster Whisperを使用する場合

In [ ]:
# whisper large-v2をhugging faceから取得して、ctranslate2でC++に変換&パラメータなどの量子化する
from faster_whisper import WhisperModel
!ct2-transformers-converter --model openai/whisper-large-v2 --output_dir whisper-large-v2-ct2 \
    --copy_files tokenizer.json --quantization float16

model_path = "whisper-large-v2-ct2/"

In [ ]:
# Run on GPU with FP16
model = WhisperModel(model_path, device="cuda", compute_type="float16")

In [ ]:
%%time
results, _ = model.transcribe(file_name,
                              language="ja")

CPU times: user 144 ms, sys: 20.3 ms, total: 165 ms
Wall time: 442 ms


In [ ]:
create_srt_file(file_name="transcribe_faster_whisper", results=results, fast_whisper=True)

下記は本家whisperのlarge-v2を使用する場合

In [3]:
# 本家whisperのlarge-v2を使用する場合

In [4]:
!pip install git+https://github.com/nyanta012/whisper_v2.git@adjust_segment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/nyanta012/whisper_v2.git (to revision adjust_segment) to /tmp/pip-req-build-4opr1xb7
  Running command git clone --filter=blob:none --quiet https://github.com/nyanta012/whisper_v2.git /tmp/pip-req-build-4opr1xb7
  Running command git checkout -b adjust_segment --track origin/adjust_segment
  Switched to a new branch 'adjust_segment'
  Branch 'adjust_segment' set up to track remote branch 'adjust_segment' from 'origin'.
  Resolved https://github.com/nyanta012/whisper_v2.git to commit 6abd9e5ac8d70db044222bb7f719b2b77cdbf732
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.0 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=796969 sha256=91af63bff534d962bb0f6913

In [5]:
import whisper

model = whisper.load_model("large-v2")

100%|██████████████████████████████████████| 2.87G/2.87G [00:24<00:00, 128MiB/s]


In [6]:
file_name = "サンプル動画.mkv"

In [9]:
%%time
segments = model.transcribe(file_name, language="ja", segment_length_ratio=1)

CPU times: user 5.74 s, sys: 149 ms, total: 5.89 s
Wall time: 6.12 s


In [10]:
create_srt_file(file_name="transcribe_openai_whisper_ja", results=segments["segments"], fast_whisper=False)